# Structured Data Matching Exemplar

xxx

Semantic text similarity techniques (W2V etc) may not be quite a fit for this, even if we treat a record like a string, does knowledge of context even help? with this in mind, it is not used in this notebook.

## Key Terminology

The three primary tasks involved in entity resolution are deduplication, record linkage, and canonicalization:

**Deduplication**: For a given CSV containing N rows and N columns, identify the rows that are very likely duplicates with another row. For example, a Bob Smith vs Robert Smith, using all the data avaliable to support that process.

**Record Linkage**: For two given CSV's, attempt to identify which rows belong to the same entity and join the rows together, even if they use entirely different columns.

**Canonicalization**: converting data with more than one possible representation into a standard form.

### Load Data

In [23]:
import recordlinkage
from recordlinkage.datasets import load_febrl1
df = load_febrl1()
df.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [27]:
# Reset index into a column
df.reset_index(level=0, inplace=True)

### Technique 1 - Fuzzy and String Matching

You can use:
- Edit distance.
- NGRAMS.

In [62]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

pd.set_option('display.max_colwidth', -1)

c:\users\jorda\desktop\machine-learning-blueprints\env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


fuzzy string matching is the technique of finding strings that match a pattern approximately (rather than exactly). In another word, fuzzy string matching is a type of search that will find matches even when users misspell words or enter only partial words for the search. It is also known as approximate string matching.

In [63]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Combine Columns into a single string...

In [64]:
df = df.replace(np.nan, '', regex=True)

In [65]:
df["combined"] = df["given_name"] + ' ' + df["surname"] + ' ' + df["address_1"] + ' ' + df["address_2"] + ' ' + df["state"]

In [66]:
df["combined"].head()

0     waller tullaroop street willaroo wa           
1    lachlan berry giblin street killarney qld      
2    deakin sondergeld goldfinch circuit kooltuo vic
3    kayla harrington maltby circuit coaling nsw    
4    luke purdon ramsay place mirani vic            
Name: combined, dtype: object

Check an input against the raw string list in the dataset of records.

In [67]:
print(process.extractOne('luke purdon ramsay mirani willaroo', df["combined"]))

('luke   calala farm vic', 86, 477)


This approach has clear limits, as it essentially focuses on an overlap of characters in determining duplication, it does nto account for feilds and treats all characters as equal in a merge. This approach is also very slow on a large set.

### Technique 2 - Record Linkage Library - Deduplication.

With the method index, all possible (and unique) record pairs are made. The method returns a pandas.MultiIndex. This is called indexing.

In [8]:
indexer = recordlinkage.Index()
indexer.full()

candidate_links = indexer.index(df)
print (len(df), len(candidate_links))

1000 499500


Now, every record has a unique pair with each other record.

#### Blocking

One of the most well known indexing methods is named blocking. This method includes only record pairs that are identical on one or more stored attributes of the person (or entity in general). The blocking method can be used in the recordlinkage module.

In this case, we are saying that we want the given name to match, before trying to dedup, reducing volume, giving duplication is reasonably rare in structured data.

a technique called 'Sorted Neighbourhood Indexing' is more advanced and can be used instead.

In [10]:
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(df)

print (len(candidate_links))

2082


In [11]:
candidate_links

MultiIndex([('rec-183-dup-0',   'rec-122-org'),
            (  'rec-248-org',   'rec-122-org'),
            (  'rec-248-org', 'rec-183-dup-0'),
            ('rec-122-dup-0',   'rec-122-org'),
            ('rec-122-dup-0', 'rec-183-dup-0'),
            ('rec-122-dup-0',   'rec-248-org'),
            (  'rec-469-org',   'rec-122-org'),
            (  'rec-469-org', 'rec-183-dup-0'),
            (  'rec-469-org',   'rec-248-org'),
            (  'rec-469-org', 'rec-122-dup-0'),
            ...
            ('rec-407-dup-0',   'rec-407-org'),
            ('rec-367-dup-0',   'rec-367-org'),
            ('rec-103-dup-0',   'rec-103-org'),
            ('rec-195-dup-0',   'rec-195-org'),
            ('rec-184-dup-0',   'rec-184-org'),
            (  'rec-252-org', 'rec-252-dup-0'),
            ( 'rec-48-dup-0',    'rec-48-org'),
            ('rec-298-dup-0',   'rec-298-org'),
            (  'rec-282-org', 'rec-282-dup-0'),
            (  'rec-327-org',   'rec-411-org')],
           names=['rec_

#### Compare the Records

This approach looks for both fuzzy matches and exact string matches across the feilds.

In [13]:
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, df)

In [14]:
features.head(10)

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-183-dup-0 rec-122-org             1      0.0              0       0   
rec-248-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
rec-122-dup-0 rec-122-org             1      1.0              1       1   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
rec-469-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
              rec-122-dup-0           1      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-183-dup-0 rec-122-org        0        0.0  
rec-248-org   rec-122-org        1        0.0  
              rec-183-dup-0      0        0.0  
rec-122-dup-0 rec-122-org        1        1.0  
              rec-183-dup-0      0        0.0  
              rec-248-org        1        0.0  
rec-469-org   rec-122-org        0        0.0  
              rec-183-dup-0      1        0.0  
              rec-248-org        0        0.0  
              rec-122-dup-0      0        0.0

In [15]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

In [16]:
matches = features[features.sum(axis=1) > 3]

print(len(matches))
matches.head(10)

317


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-122-dup-0,rec-122-org,1,1.0,1,1,1,1.0
rec-183-org,rec-183-dup-0,1,1.0,1,1,1,1.0
rec-248-dup-0,rec-248-org,1,1.0,1,1,1,1.0
rec-373-dup-0,rec-373-org,1,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,1,1.0,1,1,1,1.0
rec-342-dup-0,rec-342-org,1,1.0,0,1,1,1.0
rec-397-org,rec-397-dup-0,1,1.0,1,1,1,0.0
rec-472-org,rec-472-dup-0,1,1.0,1,1,1,0.0
rec-330-org,rec-330-dup-0,1,0.0,1,1,1,0.0


You can see, that the result is a basic count if the matching criteria is met and we chose an arbitrary boundary if enough conditions are met. 

In [33]:
df[df['rec_id'] == 'rec-342-dup-0']

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
827,rec-342-dup-0,kayla,honeychurch,NaN,sturt avenue,dimenttowers,ashgrove,2405,vic,19140727,7800242


In [34]:
df[df['rec_id'] == 'rec-342-org']

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
589,rec-342-org,kayla,honeychurch,102,sturt avenue,diment towers,ashgrove,2450,vic,19151022,7800242


### Technique 3 - tf-idf

In [68]:
import re
def ngrams(string, n=3):
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "Department":')
print(ngrams('Department'))

All 3-grams in "Department":
[' De', 'Dep', 'epa', 'par', 'art', 'rtm', 'tme', 'men', 'ent', 'nt ']


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(df["combined"])

In [96]:
from sklearn.metrics.pairwise import cosine_similarity


input_a = 'kayla honeychurch sturt avenue dimenttowers vic'
input_b = 'kayla honeychurch sturt avenue diment towers vic'
input_c = 'jordan mcdonald mount vernon diment towers na'


print(cosine_similarity(vectorizer.transform([input_a]), vectorizer.transform([input_b])))
print(cosine_similarity(vectorizer.transform([input_a]), vectorizer.transform([input_c])))

[[0.9001851]]
[[0.20512969]]


In [183]:
def get_matches_df(sparse_matrix, name_vector):
    
    left_side = []
    right_side = []
    similairity = []
    
    for i in range(0, 5):
        print(name_vector[i])
        for j in range(0, len(name_vector)):
            left_side.append(name_vector[i])
            right_side.append(name_vector[j])
            similairity.append(sparse_matrix[i][j])
    
    return pd.DataFrame({'left_side': left_side,
                      'right_side': right_side,
                       'similairity': similairity})

In [184]:
matches_df = get_matches_df(cosine_similarity(tf_idf_matrix), df["combined"])

 waller tullaroop street willaroo wa
lachlan berry giblin street killarney qld
deakin sondergeld goldfinch circuit kooltuo vic
kayla harrington maltby circuit coaling nsw
luke purdon ramsay place mirani vic


In [185]:
matches_df.sample(20)

,left_side,right_side,similairity
1220,lachlan berry giblin street killarney qld,joshus maclennan karsjl roper nplace qld,0.036297
28,waller tullaroop street willaroo wa,dakota morrison longmore crescent three acres nsw,0.003779
2044,deakin sondergeld goldfinch circuit kooltuo vic,emma green learmonth drive wallace heights qld,0.006336
548,waller tullaroop street willaroo wa,gerste booth cre scent donpre nsw,0.000000
1569,lachlan berry giblin street killarney qld,ben browne tillyard drive willow wood vic,0.057225
3190,kayla harrington maltby circuit coaling nsw,darcy jolly antill street agar abi nsw,0.014108
4923,luke purdon ramsay place mirani vic,kirah mccarthy laseron place wa,0.106749
3123,kayla harrington maltby circuit coaling nsw,alexa-rose campblel neeld place qld,0.000000
2548,deakin sondergeld goldfinch circuit kooltuo vic,gerste booth cre scent donpre nsw,0.023788
481,waller tullaroop street willaroo wa,jacob walls atherton street indra nsw,0.106403


In [199]:
matches_df[matches_df['similairity'] > 0.7]

,left_side,right_side,similairity
0,waller tullaroop street willaroo wa,waller tullaroop street willaroo wa,1.000000
474,waller tullaroop street willaroo wa,jamilla wallner tullaroop street willaroo wa,0.836462
1001,lachlan berry giblin street killarney qld,lachlan berry giblin street killarney qld,1.000000
1330,lachlan berry giblin street killarney qld,lachlan berry giblin street killarney qld,1.000000
2002,deakin sondergeld goldfinch circuit kooltuo vic,deakin sondergeld goldfinch circuit kooltuo vic,1.000000
2351,deakin sondergeld goldfinch circuit kooltuo vic,deakin sondergeld goldfinch circuit kooltuo vic,1.000000
3003,kayla harrington maltby circuit coaling nsw,kayla harrington maltby circuit coaling nsw,1.000000
3290,kayla harrington maltby circuit coaling nsw,kayla harrington maltby circuit coaling nsw,1.000000
4004,luke purdon ramsay place mirani vic,luke purdon ramsay place mirani vic,1.000000
4333,luke purdon ramsay place mirani vic,mia purdon ramsay place mirani vic,0.880569


## Resources

https://docs.dedupe.io/en/latest/How-it-works.html
- A promising library for record linkage and deduplication, alongside a useful walthrough of the key ideas involved in this link.
https://recordlinkage.readthedocs.io/en/latest/notebooks/data_deduplication.html
- Another package